# NEQR Quantum Circuit Example

**Author: Luis Jose Mantilla Santa Cruz**

In this notebook, we will implement a quantum circuit for the Novel Enhanced Quantum Representation (NEQR) model using Qiskit. We will demonstrate how to encode a small 2x2 grayscale image into a quantum state.

## Reference

For more details on NEQR, please refer to the following article:

- **Title:** NEQR: a novel enhanced quantum representation of digital images
- **Authors:** [Include author names if available]
- **Published:** 26 March 2013
- **Journal:** Quantum Information Processing
- **Volume:** 12, Pages 2833–2860
- **Link:** [NEQR Article on SpringerLink](https://link.springer.com/article/10.1007/s11128-013-0567-z)

## Step 1: Import Libraries

First, we need to import the required libraries from Qiskit and other useful packages.


In [ ]:
# Import necessary libraries
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram

## Step 2: Define the Quantum Circuit

We'll define a quantum circuit for encoding a 2x2 image with a grayscale range of 2. This will use 4 qubits: 2 for the coordinates and 2 for the grayscale values.


In [ ]:
# Define the quantum circuit
n = 2  # Number of qubits for coordinates
q = 1  # Number of qubits for grayscale values
qc = QuantumCircuit(n + q, n + q)


## Step 3: Encode Pixel Values

We will encode grayscale values and coordinates into the quantum circuit. For simplicity, we assume the pixel values are as follows:

- \( f(0,0) = 0 \)
- \( f(0,1) = 1 \)
- \( f(1,0) = 1 \)
- \( f(1,1) = 0 \)

The function `encode_pixel_value` sets the qubits based on these values.


In [ ]:
def encode_pixel_value(qc, value, row, col):
    """ Encode a grayscale value into the circuit """
    if value == 1:
        qc.x(qc.qubits[row])
    # Encode coordinates
    if col == 1:
        qc.x(qc.qubits[n + row])
    if row == 1:
        qc.x(qc.qubits[n + col])


## Step 4: Encode Each Pixel

Now, we use the `encode_pixel_value` function to encode the values for each pixel.


In [ ]:
# Encode each pixel
encode_pixel_value(qc, 0, 0, 0)  # f(0,0) = 0
encode_pixel_value(qc, 1, 0, 1)  # f(0,1) = 1
encode_pixel_value(qc, 1, 1, 0)  # f(1,0) = 1
encode_pixel_value(qc, 0, 1, 1)  # f(1,1) = 0


## Step 5: Apply Hadamard Gates

We will apply Hadamard gates to the qubits to put them into a superposition state.


In [ ]:
# Apply Hadamard gates to create superposition
qc.h(range(n + q))


## Step 6: Entangle Qubits

We use CNOT gates to entangle the coordinate qubits with the grayscale value qubits.


In [ ]:
# Apply CNOT gates to entangle the qubits
for i in range(n):
    qc.cx(i, n + i)


## Step 7: Measurement

We will measure all qubits to get the results.


In [ ]:
# Measurement
qc.measure(range(n + q), range(n + q))

# Draw the quantum circuit
qc.draw()


## Step 8: Simulate the Circuit

We will use a quantum simulator to execute the circuit and obtain the results.


In [ ]:
# Simulate the circuit
backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=1024)
result = job.result()
counts = result.get_counts()

# Plot the results
plot_histogram(counts).show()


## Conclusion

In this notebook, we implemented a quantum circuit for the NEQR model using Qiskit. We encoded a simple 2x2 grayscale image into the quantum state and visualized the results.

The resulting quantum state allows for efficient representation and processing of image data in a quantum system.
